# Lab | API wrappers

In [92]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [93]:
# Getting the playlists from this website "The Best Spotify Playlists in 2021"
url = "https://audiohype.io/resources/the-best-spotify-playlists/"
response = requests.get(url)
response.status_code 

200

In [94]:
soup = BeautifulSoup(response.content, "html.parser")

In [95]:
# I read 29 playlists ids
playlists_ids = []
num_iter = len(soup.select("h3"))
for i in range(num_iter):
    temp = soup.select("h3 a")[i]["href"]
    playlists_ids.append(temp.split('playlist/')[1].split('?')[0])
playlists_ids


['37i9dQZF1DXcBWIGoYBM5M',
 '31ymdYCITDnZRtkKzP3Itp',
 '0cc8YMQWsSzODyTpdVB6mI',
 '37i9dQZF1DX5hHfOi73rY3',
 '37i9dQZF1DXa71eg5j9dKZ',
 '4kw9kdjzx1UmyWvpysl0y2',
 '1xNWydoMmA8210KeHU948w',
 '6vDGVr652ztNWKZuHvsFvx',
 '6J1r02xyO2qkMA9dDNZytJ',
 '5wtqmpRl17iVz2nW8U6njL',
 '0NCspsyf0OS4BsPgGhkQXM',
 '19y0UVk0bcrJWEqMwBHosj',
 '4oOZJEq1TBUti6PSouTo5M',
 '068WHS0zOWsqvn2uIBYb5D',
 '6ZTpgxK6BT92mmsqwETj9l',
 '37i9dQZF1DWSvKsRPPnv5o',
 '37i9dQZF1DXe4Cw8IKKIvr',
 '37i9dQZF1DXcDoDDetPsEg',
 '65xSncKQzG6Suseh5gfYP1',
 '0MSCX9tZWQmitMQsfhvZIl',
 '1PmwMQY86pJuAm7veFt3u2',
 '1cwmC5BQ6IIuLsbyZNBIJl',
 '6wjCvkAFovrVIRM8VfZLZG',
 '37i9dQZF1DWVV27DiNWxkR',
 '37i9dQZF1DWVzZlRWgqAGH',
 '37i9dQZF1DXc8kgYqQLMfH',
 '31FWVQBp3WQydWLNhO0ACi',
 '37i9dQZF1DXa9wYJr1oMFq',
 '37i9dQZF1DX8FwnYE6PRvL']

In [96]:
secrets_file = open("secrets.txt","r")
string = secrets_file.read()
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

In [97]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

In [98]:
def get_name_artists_from_track(track):
    return [(track["name"],artist["name"],track["uri"],sp.audio_features(track["uri"])) for artist in track["artists"]]

def get_name_artists_from_playlist_item(playlist_item):
    return get_name_artists_from_track(playlist_item['track'])

def get_name_artists_from_playlist(playlist):
    return list(map(get_name_artists_from_playlist_item, playlist['items']))

def flatten(input_list):
    return [item for sublist in input_list for item in sublist]

In [99]:
# Reading all 29 playlists and collecting title, artist and audio features for each track

from random import randint
from time import sleep

all_songs = []
for i in range(len(playlists_ids)):
        playlist = sp.user_playlist_tracks("spotify", playlists_ids[i])
        try:
            all_songs += (flatten(get_name_artists_from_playlist(playlist)))
        except:
            print("Playlist: ", playlists_ids[i], "<-- There is an error with this playlist:")
        sleep(randint(1,3))

Playlist:  5wtqmpRl17iVz2nW8U6njL <-- There is an error with this playlist:


In [100]:
len(all_songs)

3215

In [101]:
all_songs[0]

('Easy On Me',
 'Adele',
 'spotify:track:0gplL1WMoJ6iYaPgMCL0gX',
 [{'danceability': 0.604,
   'energy': 0.366,
   'key': 5,
   'loudness': -7.519,
   'mode': 1,
   'speechiness': 0.0282,
   'acousticness': 0.578,
   'instrumentalness': 0,
   'liveness': 0.133,
   'valence': 0.13,
   'tempo': 141.981,
   'type': 'audio_features',
   'id': '0gplL1WMoJ6iYaPgMCL0gX',
   'uri': 'spotify:track:0gplL1WMoJ6iYaPgMCL0gX',
   'track_href': 'https://api.spotify.com/v1/tracks/0gplL1WMoJ6iYaPgMCL0gX',
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0gplL1WMoJ6iYaPgMCL0gX',
   'duration_ms': 224695,
   'time_signature': 4}])

In [102]:
db_songs = pd.DataFrame(all_songs, columns=['title', 'artist', 'uri', 'audio'])
db_songs
split_df = pd.DataFrame(db_songs['audio'].tolist(), columns=['audio'])['audio'].apply(pd.Series)
db_songs= pd.concat([db_songs, split_df.drop(['uri'], axis=1)], axis=1).drop('audio', axis=1)

In [103]:
db_songs

,title,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Easy On Me,Adele,spotify:track:0gplL1WMoJ6iYaPgMCL0gX,0.604,0.366,5.0,-7.519,1.0,0.0282,0.578000,0.000000,0.1330,0.130,141.981,audio_features,0gplL1WMoJ6iYaPgMCL0gX,https://api.spotify.com/v1/tracks/0gplL1WMoJ6i...,https://api.spotify.com/v1/audio-analysis/0gpl...,224695.0,4.0
1,STAY (with Justin Bieber),The Kid LAROI,spotify:track:5HCyWlXZPP0y6Gqq8TgA20,0.591,0.764,1.0,-5.484,1.0,0.0483,0.038300,0.000000,0.1030,0.478,169.928,audio_features,5HCyWlXZPP0y6Gqq8TgA20,https://api.spotify.com/v1/tracks/5HCyWlXZPP0y...,https://api.spotify.com/v1/audio-analysis/5HCy...,141806.0,4.0
2,STAY (with Justin Bieber),Justin Bieber,spotify:track:5HCyWlXZPP0y6Gqq8TgA20,0.591,0.764,1.0,-5.484,1.0,0.0483,0.038300,0.000000,0.1030,0.478,169.928,audio_features,5HCyWlXZPP0y6Gqq8TgA20,https://api.spotify.com/v1/tracks/5HCyWlXZPP0y...,https://api.spotify.com/v1/audio-analysis/5HCy...,141806.0,4.0
3,All Too Well (10 Minute Version) (Taylor's Ver...,Taylor Swift,spotify:track:5enxwA8aAbwZbf5qCHORXi,0.631,0.518,0.0,-8.771,1.0,0.0303,0.274000,0.000000,0.0880,0.205,93.023,audio_features,5enxwA8aAbwZbf5qCHORXi,https://api.spotify.com/v1/tracks/5enxwA8aAbwZ...,https://api.spotify.com/v1/audio-analysis/5enx...,613027.0,4.0
4,Cold Heart - PNAU Remix,Elton John,spotify:track:6zSpb8dQRaw0M1dK8PBwQz,0.796,0.798,1.0,-6.312,1.0,0.0317,0.034000,0.000042,0.0952,0.942,116.032,audio_features,6zSpb8dQRaw0M1dK8PBwQz,https://api.spotify.com/v1/tracks/6zSpb8dQRaw0...,https://api.spotify.com/v1/audio-analysis/6zSp...,202735.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3210,Somebody Told Me,The Killers,spotify:track:7HGTkn2aX7MNdKs7nV2xBt,0.509,0.987,10.0,-3.198,0.0,0.0886,0.000081,0.000928,0.1190,0.647,138.039,audio_features,7HGTkn2aX7MNdKs7nV2xBt,https://api.spotify.com/v1/tracks/7HGTkn2aX7MN...,https://api.spotify.com/v1/audio-analysis/7HGT...,197160.0,4.0
3211,You Really Got Me,The Kinks,spotify:track:4D0BiMiLN6z2x7Tq9BcqoT,0.576,0.963,1.0,-3.950,1.0,0.1020,0.192000,0.000000,0.0870,0.961,137.331,audio_features,4D0BiMiLN6z2x7Tq9BcqoT,https://api.spotify.com/v1/tracks/4D0BiMiLN6z2...,https://api.spotify.com/v1/audio-analysis/4D0B...,133813.0,4.0
3212,I Was Made For Lovin' You,KISS,spotify:track:2ZSCy3P1QzpJySCWA6NRIU,0.768,0.852,4.0,-6.215,1.0,0.0380,0.213000,0.000074,0.0723,0.867,128.374,audio_features,2ZSCy3P1QzpJySCWA6NRIU,https://api.spotify.com/v1/tracks/2ZSCy3P1QzpJ...,https://api.spotify.com/v1/audio-analysis/2ZSC...,271240.0,4.0
3213,Golden Skans,Klaxons,spotify:track:3aFSo24NwIQHeO3aJdyJb9,0.451,0.852,2.0,-3.409,0.0,0.0418,0.000652,0.000003,0.2510,0.601,141.935,audio_features,3aFSo24NwIQHeO3aJdyJb9,https://api.spotify.com/v1/tracks/3aFSo24NwIQH...,https://api.spotify.com/v1/audio-analysis/3aFS...,165946.0,4.0


In [104]:
db_songs.isna().sum()

title               0
artist              0
uri                 0
danceability        2
energy              2
key                 2
loudness            2
mode                2
speechiness         2
acousticness        2
instrumentalness    2
liveness            2
valence             2
tempo               2
type                2
id                  2
track_href          2
analysis_url        2
duration_ms         2
time_signature      2
dtype: int64

In [105]:
# Deleting songs with NaN values

db_songs = db_songs[~db_songs['mode'].isnull()].reset_index(drop=True)

In [106]:
# for each artist in my list of songs, I collect 25 songs

all_songs = []
for artist in db_songs['artist'].unique():
    results = sp.search(q=artist, limit=25)
    for i in range(0, len(results['tracks']['items'])):
        if (len(results)>0):
            try:
                all_songs += (get_name_artists_from_track(results['tracks']['items'][i]))
            except:
                continue
    sleep(randint(1,3))


KeyboardInterrupt: 

In [114]:
db_songs = pd.DataFrame(all_songs, columns=['title', 'artist', 'uri', 'audio'])
split_df = pd.DataFrame(db_songs['audio'].tolist(), columns=['audio'])['audio'].apply(pd.Series)
db_songs= pd.concat([db_songs, split_df.drop(['uri'], axis=1)], axis=1).drop('audio', axis=1)

In [115]:
# Deleting songs with NaN values

db_songs = db_songs[~db_songs['mode'].isnull()].reset_index(drop=True)

In [116]:
import pickle

with open("db_songs.txt", "wb") as f:   #Pickling
    pickle.dump(db_songs, f)